In [0]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn

from torchvision import datasets, transforms

In [0]:
torch.cuda.is_available() # mengecek cude apakah sudah aktif  atau belum 

True

In [0]:
#NEURAL NETWORK
class MLP(nn.Module):
	def __init__(self, in_feats, out_feats):
		super(MLP, self).__init__()

		layer_1 = [
			nn.Linear(in_feats, in_feats*2),
			nn.Sigmoid()
		]

		layer_2 = [
			nn.Linear(in_feats*2, in_feats),
			nn.Sigmoid()
		]
    
		layer_3 = [
			nn.Linear(in_feats, out_feats),
			nn.Softmax()
		]

		self.nets = nn.Sequential(*layer_1, *layer_2, *layer_3)

	def forward(self, x):
		out = self.nets(x);
		return out

In [0]:
x = np.arange(3*4).reshape(3,4)
x = torch.Tensor(x)
net = MLP(4,1)
net(x)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:100: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


tensor([[1.],
        [1.],
        [1.]], grad_fn=<SoftmaxBackward>)

In [0]:
BATCH_SIZE =37
EPOCH = 80
LR = 0.005
MOMENT = 0.9
EVAL_STEP = 2

In [0]:
trans = transforms.Compose([
	transforms.ToTensor(),
	transforms.Normalize((0.0,),(1.0,))
])

train_set = datasets.MNIST(
	root = '', train = True,
	transform = trans, download = True
)

test_set = datasets.MNIST(
	root = '', train = False,
	transform = trans, download = True
)

train_loader = torch.utils.data.DataLoader(
	dataset = train_set, batch_size = BATCH_SIZE,
	shuffle = True
)

test_loader = torch.utils.data.DataLoader(
	dataset = test_set, batch_size = BATCH_SIZE,
	shuffle = False
)

print(len(train_loader), len(test_loader))

Extracting MNIST/raw/train-images-idx3-ubyte.gz to MNIST/raw


Extracting MNIST/raw/train-labels-idx1-ubyte.gz to MNIST/raw



Extracting MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST/raw


Extracting MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST/raw
Processing...
Done!
1622 271


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


In [0]:
net = MLP (784,10)
net = net.cuda()

optimizer = optim.SGD(net.parameters(), lr = LR, momentum = MOMENT)
criterion = nn.CrossEntropyLoss()

In [0]:
def train(net, criterion, optim, loader, epoch):
	net.train()
	total_loss = 0;
	for data, label in loader:
		batch_size = data.size(0)

		optim.zero_grad()
		data, label = data.cuda(), label.cuda()
		data = data.view(batch_size, -1)
		out = net(data)
		loss = criterion(out, label)

		total_loss += loss.item()

		loss.backward()
		optim.step()

	print("[%03d/%03d] avg loss = %0.4f" %(epoch, EPOCH, total_loss/len(loader)))

def test(net, loader):
	net.eval()
	pred = []
	target = []
	for data, label in loader:
		batch_size = data.size(0)
		data = data.view(batch_size, -1)
		data = data.cuda()

		pred_batch = net(data)
		pred_batch = pred_batch.to('cpu').detach().numpy()
		pred_batch = np.argmax(pred_batch, 1)

		pred.append(pred_batch)
		target.append(label.numpy())

		pred = np.concatenate(pred)
		target = np.concatenate(target)
		acc = (np.sum(pred == target) / len(pred))

		return pred, target, acc

In [0]:
prediction = None
target = None
for epoch in range(1, EPOCH+1):
	train(net, criterion, optimizer, train_loader, epoch)
	if epoch % EVAL_STEP == 0:
		with torch.no_grad():
			prediction, target, acc = test(net, test_loader)
			print(acc)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:100: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


[001/080] avg loss = 2.3012
[002/080] avg loss = 2.2968
0.2702702702702703
[003/080] avg loss = 2.2488
[004/080] avg loss = 2.1526
0.4864864864864865
[005/080] avg loss = 2.0716
[006/080] avg loss = 1.9775
0.7027027027027027
[007/080] avg loss = 1.9121
[008/080] avg loss = 1.8747
0.7567567567567568
[009/080] avg loss = 1.8479
[010/080] avg loss = 1.8306
0.8378378378378378
[011/080] avg loss = 1.8192
[012/080] avg loss = 1.8117
0.8378378378378378
[013/080] avg loss = 1.7937
[014/080] avg loss = 1.7677
0.8108108108108109
[015/080] avg loss = 1.7570
[016/080] avg loss = 1.7493
0.8108108108108109
[017/080] avg loss = 1.7430
[018/080] avg loss = 1.7376
0.8108108108108109
[019/080] avg loss = 1.7328
[020/080] avg loss = 1.7289
0.8108108108108109
[021/080] avg loss = 1.7255
[022/080] avg loss = 1.7221
0.8108108108108109
[023/080] avg loss = 1.7194
[024/080] avg loss = 1.7169
0.8108108108108109
[025/080] avg loss = 1.7149
[026/080] avg loss = 1.7129
0.8378378378378378
[027/080] avg loss = 1.71